In [5]:
import os 
import pickle as pkl
import pandas as pd
import numpy  as np

import matplotlib.pyplot as plt

from sklearn.metrics import r2_score
from autoemulate.compare import AutoEmulate

from sklearn.model_selection import KFold

from sklearn.preprocessing import RobustScaler

def main():
    

    path_X  = '/Users/pmzff/Documents/GitHub/ModularCircFF/ExploreModularCirc/Input/input_2000.csv'
    path_y = '/Users/pmzff/Documents/GitHub/ModularCircFF/ExploreModularCirc/Outputs/Output_2000/PCA/PC3.csv'
    
    N_cases = 2000
    X_df  = pd.read_csv(path_X)
    y_df  = pd.read_csv(path_y)
    #  Select relevant inputs only
    relevant_columns = []
    for col in X_df.columns:
        relevant_columns.append(col)
        if col == 'T': break

    # Select only first 5 inputs 
    filtered_input = X_df[relevant_columns]

    best_emulator_list = []

           
    for col in y_df.columns: 
        if col == 'dt': continue   
        print('\n'*2)
        print(f'Col {col}\n')
        ae = AutoEmulate()
        model_subset = [
            "NeuralNetSk", 
            # "SecondOrderPolynomial", 
            # "SupportVectorMachines", 
            # "GradientBoosting", 
            # "RandomForest"
        ]
        ae.setup(filtered_input .values, 
                y_df[col].values, 
                models=model_subset, 
                n_jobs=5, 
                param_search=True,
                scale=RobustScaler(),
                param_search_type='random',  # bayes, random
                param_search_iters=20,
                cross_validator=KFold(n_splits=5, shuffle=True),
                log_to_file=True
                )
        best_model = ae.compare() 
        
        print(ae.summarise_cv())
        
        best_emulator = ae.refit(best_model) 
        
        print(best_emulator.named_steps['model'].get_params())
    
        os.system(f'mkdir -p Auto_Emulators')
        ae.save(best_emulator, f'Auto_Emulators/emulator_{N_cases}_{col}.pkl')    
    return 



In [6]:
main()




Col PC3



,Values
Simulation input shape (X),"(2000, 22)"
Simulation output shape (y),"(2000,)"
Proportion of data for testing (test_set_size),0.2
Scale input data (scale),RobustScaler()
Scaler (scaler),StandardScaler
Do hyperparameter search (param_search),True
Type of hyperparameter search (search_type),random
Number of sampled parameter settings (param_search_iters),20
Reduce dimensionality (reduce_dim),False
Cross validator (cross_validator),KFold


Initializing:   0%|          | 0/1 [00:00<?, ?it/s]

         model short      rmse        r2
0  NeuralNetSk   nns  0.363291  0.882352
{'activation': 'relu', 'alpha': 7.820240077460717e-05, 'hidden_layer_sizes': (50,), 'learning_rate': 'constant', 'learning_rate_init': 0.0003229772627793101, 'max_iter': 200, 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001}


In [7]:
import pickle

# File path
file_path = "/Users/pmzff/Documents/GitHub/ModularCircFF/Emulation/Auto_Emulators/emulator_2000_PC2.pkl"

# Open and read the pickle file
with open(file_path, 'rb') as file:
    data = pickle.load(file)



UnpicklingError: invalid load key, '\x0e'.